# 3, AR 1 simulation
import numpy as np

def ar1_simulate(rho, errors, y0):

    y = []
    y.append(y0)
    t = len(errors)
    for i in range(1,t):
        y.append(y[-1] * rho + errors[i-1])
    
    y = np.asarray(y)
  
    return y



In [789]:
# Marie code
def ar(rho,errors,y0):
    t = errors.shape[0]
    y = np.zeros(t)
    y[0] = y0

    for i in range(1,t):
        y[i] = rho*y[i-1] + errors[i-1]
    return y


def sub_rv(rets,k):
    n = rets.shape[0]
    total_sum = 0.0
    for j in range(0,n-k+1):
        inside_sum = 0.0
        for i in range(0,k):
            inside_sum += rets[j + i -1]
        total_sum += (inside_sum)**2
    rv_ss = ((n/k)/(n-k+1))*total_sum
    return rv_ss

## aish code
import pandas as pd
import numpy as np


def weighted_hs(rets, lam, window, p):
        
    def single(rets, lam, p):
        rets = np.asarray(rets)
        t = rets.shape[0]
        w = lam * np.arange(t)[::-1] * (1 - lam) / (1 - lam * t)
        idx = np.argsort(rets)
        cw = np.cumsum(w[idx])
        print(cw)
        loc = np.amin(np.argwhere(cw > p))
        return rets[idx][loc]
    
    t = rets.shape[0]
    out = {}
    for i in range(window, t+1):
        out[rets.index[i-1]] = single(rets.iloc[i - window : i], lam, p)
    out_rets = pd.Series(out)
    return -out_rets.reindex(rets.index)


# 2 Subsampled RV
import pandas as pd
def subsampled_rv(rets,k):
    n = len(rets)

    # k is the block size:
    r2 = []
    for i in range(k):
        r2.append((rets.iloc[i::k] ** 2).sum())
    
    r2 = pd.Series(r2).sum()
    num = n / k
    denom = n - k + 1
    multiplier = num / denom

    rv = multiplier * r2
    rv = float(rv)

    return rv






def weighted_hs(rets, lam, window, p):
    wT = (1 - lam) / (1 - lam ** window)

    wt = np.zeros(window)
    wt[window-1] = wT


    for i in range(window-2,0,-1):
        wt[i] = wt[i+1] * lam
    
    n = rets.shape[0]
    var = np.empty(n)
    var.fill(np.nan)
    save = {}

    for i in range(window,n):
        modeled_rets = rets[i-window:i]
        modeled_rets.index = range((window))
        modeled_rets = pd.DataFrame([modeled_rets,wt]).T
        modeled_rets.columns = ["rets","weights"]
        modeled_rets = modeled_rets.sort_values("rets")

        cutoff = int(window * p) + 1
        modeled_rets["rets"][cutoff:] = 0

        sums = (modeled_rets["rets"] * modeled_rets["weights"]).sum()

        var[i-1] = sums
        save[i] = modeled_rets
    
    var = pd.Series(var)
    var.index = rets.index

    return var



    
    





In [909]:
np.zeros(2)

array([0., 0.])

In [1028]:
### MY CODE

import pandas as pd
import numpy as np

def weighted_hs(rets, lam, window, p):
    wT = (1 - lam) / (1 - lam ** window)

    wt = np.zeros(window)
    wt[window-1] = wT

    for t in range(0,window-1):
        wt[t] = wT * (lam ** (window - 1 - t))
    
    n = rets.shape[0]
    v = np.empty(n)
    v.fill(np.nan)

    for i in range(window,n+1):
        modeled_rets = rets[i-window:i]
        modeled_rets.index = range(window)
        modeled_rets = pd.DataFrame([modeled_rets,wt]).T
        modeled_rets.columns = ["rets","weights"]
        modeled_rets = modeled_rets.sort_values("rets")
        modeled_rets["weights"] = modeled_rets["weights"].cumsum()
        modeled_rets = np.asarray(modeled_rets)
        weights = modeled_rets[:,1]
        returns = modeled_rets[:,0]
      
   

        v[i-1] = returns[np.min(np.where(weights > p))]

    v = pd.Series(v)
    v.index = rets.index

    return v

# 2 Subsampled RV

def subsampled_rv(rets,k):
    n = len(rets)

    # k is the block size:
    r2 = []
    for i in range(0,n-k+1):
        r2.append((rets.iloc[i:k+i] ).sum() ** 2)
    
    r2 = pd.Series(r2).sum()
    num = n / k
    denom = n - k + 1
    multiplier = num / denom

    rv = multiplier * r2
    rv = float(rv)

    return rv




# 3, AR 1 simulation


def ar1_simulate(rho, errors, y0):

    y = []
    y.append(y0)
    t = len(errors)
    for i in range(1,t):
        y.append(y[-1] * rho + errors[i-1])
    
    y = np.asarray(y)
    return y



In [735]:
#anthony code 2
def subsampled_rv(rets, k):

    n = rets.shape[0]
    frac = (n/k)/(n-k+1)

    innersumsquared = pd.Series([])

    for j in range(0, n-k+1):
        subsample = rets.iloc[j:j+k]

        innersum = subsample.sum(0)
        
        innersumsquared[j] = innersum**2
        
    doublesum = innersumsquared.sum(0)    
    
    rv = frac * doublesum

    return rv

In [1015]:
import pandas_datareader as pdr
a = pdr.get_data_yahoo('AMZN',"2016-03-17","2021-03-16")['High']


weighted_hs(a, lam=0.5, window=2, p=0.05)

array([[562.33001709,   0.66666667],
       [571.40002441,   1.        ]])

In [1029]:
rg = np.random.default_rng(38218301830131)
index = pd.bdate_range("2020-01-01", periods=500, freq="D")
rets = pd.Series(rg.standard_normal(500), index=index)
lam = 0.9
window = 120
p = 0.95
weighted_hs(rets, lam=0.95, window=30, p=0.95)

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04         NaN
2020-01-05         NaN
                ...   
2021-05-10    1.814694
2021-05-11    1.814694
2021-05-12    1.814694
2021-05-13    1.814694
2021-05-14    1.814694
Freq: D, Length: 500, dtype: float64

In [990]:
testreturns = np.array([1.1, 0.2, 0.03, -0.4, 3, 3, 3, 3, 3, 3, 5, 4, -1])
testreturns = pd.Series(testreturns)

subsampled_rv(testreturns, 2)

185.59655833333335

In [991]:
a = pd.Series([1,2,1,1,5,4,6,7,2,3])


ar1_simulate(0.9, a, 1)

array([ 1.        ,  1.9       ,  3.71      ,  4.339     ,  4.9051    ,
        9.41459   , 12.473131  , 17.2258179 , 22.50323611, 22.2529125 ])